In [50]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

import pickle

In [35]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])

    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds() / 60
    df = df[(df['duration'] >  0) & (df['duration'] < 60)]

    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)

    

    df[categorical] = df[categorical].astype('category')

    return df

In [36]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')



In [37]:
df_train.head().T

,0,1,2,3,4
VendorID,2,2,2,2,2
lpep_pickup_datetime,2021-01-01 00:15:56,2021-01-01 00:25:59,2021-01-01 00:45:57,2020-12-31 23:57:51,2021-01-01 00:16:36
lpep_dropoff_datetime,2021-01-01 00:19:52,2021-01-01 00:34:44,2021-01-01 00:51:55,2021-01-01 00:04:56,2021-01-01 00:16:40
store_and_fwd_flag,N,N,N,N,N
RatecodeID,1.0,1.0,1.0,1.0,2.0
PULocationID,43,166,41,168,265
DOLocationID,151,239,42,75,265
passenger_count,1.0,1.0,1.0,1.0,3.0
trip_distance,1.01,2.53,1.12,1.99,0.0
fare_amount,5.5,10.0,6.0,8.0,-52.0


In [11]:
df_train.shape, df_val.shape

((75254, 21), (62919, 21))

In [46]:
categorical = ['PU_DO'] #'PULocationID',	'DOLocationID', 
numerical = ['trip_distance']
target = 'duration'

train_dict = df_train[categorical + numerical].to_dict(orient='records')
val_dict = df_val[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

y_train = df_train[target].values
y_val = df_val[target].values

In [47]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

print(mean_squared_error(y_val, y_pred, squared=False))

7.751201514545847


In [51]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

print(mean_squared_error(y_val, y_pred, squared=False))

7.659337943281035


In [53]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)